## KPI extraction attempt: 
#### has (v1, 2, 3 - multiple attempts met with local GPU limitations.)
#### Author : Joel Markapudi.

Regex cant understand:
- Temporal context ("compared to prior year")
- Causal relationships ("due to acquisition of...")
- Comparative statements ("increased from X to Y")
- Multi-sentence KPI narratives
- "Sales rose to $2.3B" (verb-first patterns), Table-extracted values without keywords, Abbreviated forms and industry-specific terminology.
- Projected vs actual figures, Different reporting periods in same sentence, Parent company vs subsidiary metrics, Pro-forma vs GAAP measures

```
Type A: "Revenue was $2.1B in 2018, $2.4B in 2019, and $2.8B in 2020"
Type B: "Q1 through Q4 revenues were $500M, $520M, $510M, and $530M respectively"  
Type C: "Operating margin improved from 12% to 15% while EBITDA grew from $200M to $280M"
Type D: "2019: Revenue $2.4B (up 15%), Net Income $240M (up 18%), EPS $2.40 (up 20%)"
```

#### Attempt1: leverage sota models, prod ready libs, 1M dataset which i prepped, try NER + finbert + small context model pattern.



```
Input Sentence → [Stage 1: Context Classification] → [Stage 2: Entity Extraction] 
                            ↓                                    ↓
                   (FinBERT/BART decides              (SpaCy + Patterns extract
                    if KPI-bearing)                     actual values)
                            ↓                                    ↓
                        [Stage 3: Relation Linking] ←──────────┘
                   (Connect entities to KPI types using rules + context)
                            ↓
                    [Stage 4: Validation & Normalization]
                            ↓
                      Structured KPI Record
```


1. Zero-shot classification using financial-tuned models
2. SpaCy NER - Extracts standard entities: MONEY, PERCENT, DATE, ORG, CARDINAL
3. Custom Financial Patterns ? maybe Regex patterns, and domain-specific formats: "2.3B", "15bps", "3x EBITDA"
4. Dependency Parsing. SpaCy's dependency tree to find subject-object relationships. 
5. Relation Linking. 

### Revised idea:

1. One transformer model with good questions extracts 80% of what complex pipelines would get. The model already learned the hard parts (temporal relations, financial terminology, grammatical structures) during pre-training.

```
Input Sentences → [Quick Filter] → [FinBERT QA Extraction] → [Simple Parser] → Structured KPIs
                      ↓                                            ↓
                (Skip if no numbers)                    (Already understands temporal relations)
```



In [1]:
import sys
import torch
import transformers
import polars as pl

print("=== Environment Check ===")
print(f"Python: {sys.version}")
print(f"PyTorch: {torch.__version__}")
print(f"Transformers: {transformers.__version__}")
print(f"Polars: {pl.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# transformer test
from transformers import AutoTokenizer
print("\n=== Testing Transformers ===")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
print("✅ Tokenizer loaded")


print("\n=== Testing Polars ===")
df = pl.DataFrame({"a": [1, 2, 3]})
print(f"✅ Polars DataFrame created: shape {df.shape}")

print("\n✅ All systems operational!")



c:\Users\joems\miniconda3\envs\finrag_mlops\Lib\site-packages\torch\cuda\__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


=== Environment Check ===
Python: 3.11.14 | packaged by conda-forge | (main, Oct 13 2025, 14:00:26) [MSC v.1944 64 bit (AMD64)]
PyTorch: 2.5.1
Transformers: 4.57.1
Polars: 1.34.0
CUDA available: True
GPU: NVIDIA GeForce RTX 3080 Ti Laptop GPU
GPU Memory: 17.18 GB

=== Testing Transformers ===
✅ Tokenizer loaded

=== Testing Polars ===
✅ Polars DataFrame created: shape (3, 1)

✅ All systems operational!


## Notes for learning / Recap:

1. Best Alternative: mrm8488/bert-small-finetuned-squadv2 (faster, similar accuracy) or google/flan-t5-base (can output structured text directly)
2. RoBERTa-base-squad2 is optimized for question answering
3. FinBERT is optimized for sentiment classification in financial text: Not trained for: "Extract the revenue value from this text".

### What pipeline() does internally:
    ```
    qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")`
    ```
- Is equivalent to:
    ```
    tokenizer = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2")
    model = AutoModelForQuestionAnswering.from_pretrained("deepset/roberta-base-squad2")
    ```

- Tokenize - Get model outputs - Extract answer span (first logits and last logits) - Convert tokens back to text.
- Pipeline does ALL of this with one call:
    ```
    def manual_qa(question, context):
        # Tokenize
        inputs = tokenizer(question, context, return_tensors="pt")
        
        # Get model outputs
        outputs = model(**inputs)
        
        # Extract answer span
        answer_start = torch.argmax(outputs.start_logits)
        answer_end = torch.argmax(outputs.end_logits)
        
        # Convert tokens back to text
        answer = tokenizer.decode(inputs["input_ids"][0][answer_start:answer_end+1])
        
        return answer
    ```


### BatchProcessor Design Pattern
1. BatchProcessor implements the Strategy Pattern - it encapsulates the algorithm for processing batches
```
# Why we use a separate class:
    class BatchProcessor:
        def __init__(self, qa_pipeline, batch_size=32):
            self.qa_pipeline = qa_pipeline  # Dependency injection
            self.parser = KPIParser()       # Composition
        
        def process_batch(self, sentences, metadata):
            # Encapsulates the complex batch logic
            # Makes testing easier
            # Can swap implementations
```


2. `self.parser.parse_answer(answer, sentence, metadata)`
- KPIParser.parse_answer(answer, sentence, metadata). 

3. `for index, row in df.iterrows():` is slower, we prefer now `for row in df.iter_rows(named=True):`
- named true gives dict-like and named false returns tuple-like rows.
```
    # Example:
    df = pl.DataFrame({'ticker': ['AAPL', 'MSFT'], 'value': [100, 200]})

    # Without named=True:
    for row in df.iter_rows():
        print(row)  # ('AAPL', 100), ('MSFT', 200)
        
    # With named=True:
    for row in df.iter_rows(named=True):
        print(row)  # {'ticker': 'AAPL', 'value': 100}
        print(row['ticker'])  # Direct access by name
```

In [2]:
## CODE which has - FORMATTED CELL TABLES - Opens a DOM in Jupyter with CSS styling.

from pathlib import Path
import sys, os

PROJECT_ROOT = Path.cwd().resolve()
SRC = PROJECT_ROOT / "src"
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))
# from dotenv import load_dotenv
# load_dotenv(PROJECT_ROOT / "assets" / "config.env")

%load_ext autoreload
%autoreload 2

import pandas as pd
import polars as pl
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
from itables import init_notebook_mode, show
import warnings
from IPython.display import display, HTML


def display_table_with_html(df, title=""):
    """Display pandas DataFrame as styled HTML table"""
    display(HTML(f"<h3>{title}</h3>"))
    html_str = df.to_html(classes='table table-striped table-hover', border=0)
    display(HTML(html_str))

print("Environment ready")

# # Load dataset
# DATA_PATH = Path("../data/exports/sec_filings_small_full.parquet")
# df = pl.read_parquet(DATA_PATH)

# print(f"Dataset loaded: {df.shape[0]:,} rows × {df.shape[1]} columns")
# print(f"Memory usage: {df.estimated_size('mb'):.1f} MB")

## ------------------------------------------------------------------------------------------------
## Display functions for sentences with HTML/CSS styling


from IPython.display import HTML, display
import textwrap

def display_full_sentences(df, max_rows=50, wrap_length=120, table_height="600px"):
    """
    Display dataframe with wrapped sentences in a scrollable table.
    
    Args:
        df: Polars DataFrame to display
        max_rows: Maximum rows to display (default 50)
        wrap_length: Characters per line for sentence wrapping (default 120)
        table_height: Height of scrollable area (default "600px")
    """
    
    # Define which columns to display and their widths
    display_cols = ["name", "year", "section", "sentenceID", "sentence", "likely_kpi", "has_numbers"]
    
    # Create HTML with custom CSS
    ##  ## 'Courier New', monospace;

    html_content = f"""
    <style>
        .sentence-table {{
            border-collapse: collapse;
            width: 100%;
            font-size: 12px;
            font-family: 'Consolas', monospace;
            background-color: #1e1e1e;          /* Dark background */
            color: #d4d4d4;                     /* Light gray text */
        }}
        .sentence-table th {{
            background-color: #2d2d30;          /* Dark gray header */
            color: #4EC9B0;                     /* Teal text for headers */

            padding: 8px;
            text-align: left;
            position: sticky;
            top: 0;
            z-index: 10;
        }}
        .sentence-table td {{
            border: 1px solid #ddd;
            padding: 8px;
            vertical-align: top;
        }}
        .sentence-table tr: {{
            background-color: #1e1e1e;
        }}
        .sentence-table tr:hover {{
            background-color: #2a2d2e;          /* Subtle hover effect */
            color: #ffffff;                     /* Brighter text on hover */
        }}
        .sentence-cell {{
            white-space: pre-wrap;
            word-wrap: break-word;
            max-width: 600px;
            line-height: 1.5;                    /* CHANGE LINE SPACING HERE */
            color: #d4d4d4;                     /* Light gray for sentences */
            font-weight: 400;                   /* CHANGE FONT WEIGHT HERE */
        }}
        .table-container {{
            height: {table_height};
            overflow: auto;
            border: 2px solid #4CAF50;
            position: relative;
        }}
        .kpi-true {{
            background-color: #3a3a00 !important;  /* Very dark yellow/gold */
            color: #ffeb3b !important;             /* Light yellow text for KPI rows */
        }}
        .has-numbers {{
            font-weight: bold;
            color: #569cd6;                        /* Light blue for numbers */
        }}
        .metadata {{
            color: #808080;                        /* Gray for metadata */
            font-size: 11px;
            background-color: #1e1e1e;
            padding: 5px;
        }}

        /* Custom scrollbar for better dark mode */
        .table-container::-webkit-scrollbar {{
            width: 12px;
            height: 12px;
        }}
        .table-container::-webkit-scrollbar-track {{
            background: #2d2d30;
        }}
        .table-container::-webkit-scrollbar-thumb {{
            background: #555;
            border-radius: 6px;
        }}
        .table-container::-webkit-scrollbar-thumb:hover {{
            background: #007ACC;
        }}

    </style>
    
    <div class="table-container">
        <table class="sentence-table">
            <thead>
                <tr>
    """
    
    # Add headers
    for col in display_cols:
        width = "40%" if col == "sentence" else "auto"
        html_content += f"<th style='width: {width}'>{col}</th>"
    html_content += "</tr></thead><tbody>"
    
    # Add data rows
    for i, row in enumerate(df.head(max_rows).iter_rows(named=True)):
        if i >= max_rows:
            break
            
        kpi_class = "kpi-true" if row.get('likely_kpi', False) else ""
        html_content += f"<tr class='{kpi_class}'>"
        
        for col in display_cols:
            value = row.get(col, "")
            
            if col == "sentence":
                # Wrap long sentences
                if isinstance(value, str):
                    wrapped = textwrap.fill(value, width=wrap_length)
                    html_content += f"<td class='sentence-cell'>{wrapped}</td>"
                else:
                    html_content += f"<td>{value}</td>"
            elif col in ["likely_kpi", "has_numbers"]:
                # Format boolean values
                display_val = "✓" if value else "-"
                extra_class = "has-numbers" if col == "has_numbers" and value else ""
                html_content += f"<td class='{extra_class}'>{display_val}</td>"
            elif col == "section":
                # Add section names
                section_names = {0: "Bus", 1: "Risk", 8: "MD&A", 9: "FinStmt", 10: "Notes", 11: "MktRisk"}
                section_name = section_names.get(value, str(value))
                html_content += f"<td>{value} ({section_name})</td>"
            else:
                html_content += f"<td>{value}</td>"
        
        html_content += "</tr>"
    
    html_content += f"""
            </tbody>
        </table>
    </div>
    <div class='metadata'>
        Displaying {min(max_rows, len(df))} of {len(df)} total rows | 
        Wrap: {wrap_length} chars | 
        Scroll both directions enabled
    </div>
    """
    
    display(HTML(html_content))
    
#: Simple wrap function for single column display
def display_sentences_simple(df, sentence_col="sentence", wrap_at=150):
    """
    Simpler display focusing just on sentences with metadata
    """
    html = "<div style='height: 600px; overflow-y: scroll; border: 1px solid #ccc; padding: 10px;'>"
    
    for row in df.head(100).iter_rows(named=True):
        wrapped_sentence = textwrap.fill(row[sentence_col], width=wrap_at)
        
        html += f"""
        <div style='margin-bottom: 15px; padding: 10px; background: #f9f9f9; border-left: 3px solid #4CAF50;'>
            <div style='color: #666; font-size: 11px; margin-bottom: 5px;'>
                <strong>{row['name']}</strong> | Year: {row.get('year', 'N/A')} | 
                Section: {row.get('section', 'N/A')} | ID: {row.get('sentenceID', 'N/A')}
            </div>
            <div style='font-family: monospace; font-size: 12px; white-space: pre-wrap;'>
{wrapped_sentence}
            </div>
        </div>
        """
    
    html += "</div>"
    display(HTML(html))

Environment ready


## Model Download Code

In [2]:
# Download the best models for KPI extraction
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
import torch
import time

def download_models():
    """Download and cache the best models for KPI extraction"""
    
    models_to_download = [
        ("mrm8488/bert-small-finetuned-squadv2", "QA - 42MB, fastest"),
        ("distilbert-base-cased-distilled-squad", "QA - 261MB, balanced"),
        ("deepset/roberta-base-squad2", "QA - 496MB, most accurate")
    ]
        
    print("="*60)
    print("DOWNLOADING MODELS FOR KPI EXTRACTION")
    print("="*60)
    
    for model_name, description in models_to_download:
        print(f"\n📥 Downloading: {model_name}")
        print(f"   Description: {description}")
        
        start_time = time.time()
        try:
            # Download tokenizer
            print("   - Downloading tokenizer...")
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            
            # Download model
            print("   - Downloading model weights...")
            model = AutoModelForQuestionAnswering.from_pretrained(model_name)
            
            # Move to GPU to verify it works
            if torch.cuda.is_available():
                model = model.cuda()
                print("   - Model loaded to GPU")
            
            # Test the model
            print("   - Testing model...")
            qa_pipeline = pipeline(
                "question-answering",
                model=model,
                tokenizer=tokenizer,
                device=0 if torch.cuda.is_available() else -1
            )
            
            result = qa_pipeline(
                question="What is the revenue?",
                context="The company reported revenue of $2.5 billion in 2023."
            )
            
            elapsed = time.time() - start_time
            print(f"   ✅ Success! Downloaded in {elapsed:.1f}s")
            print(f"   Test result: {result['answer']} (confidence: {result['score']:.3f})")
            
            # Clear from GPU to save memory
            if torch.cuda.is_available():
                del model
                torch.cuda.empty_cache()
            
        except Exception as e:
            print(f"   ❌ Failed: {e}")
    
    print("\n" + "="*60)
    print("DOWNLOAD COMPLETE - Models are cached for future use")
    print("="*60)
    
    # Show cache location
    from pathlib import Path
    cache_dir = Path.home() / ".cache" / "huggingface"
    print(f"\nCache location: {cache_dir}")
    
    # Show cache size
    total_size = sum(f.stat().st_size for f in cache_dir.rglob('*') if f.is_file())
    print(f"Total cache size: {total_size / 1e9:.2f} GB")

# Run the download
download_models()




DOWNLOADING MODELS FOR KPI EXTRACTION

📥 Downloading: mrm8488/bert-small-finetuned-squadv2
   Description: QA - 42MB, fastest
   - Downloading tokenizer...


c:\Users\joems\miniconda3\envs\finrag_mlops\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\joems\.cache\huggingface\hub\models--mrm8488--bert-small-finetuned-squadv2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


   - Downloading model weights...
   ❌ Failed: Due to a serious vulnerability issue in `torch.load`, even with `weights_only=True`, we now require users to upgrade torch to at least v2.6 in order to use the function. This version restriction does not apply when loading files with safetensors.
See the vulnerability report here https://nvd.nist.gov/vuln/detail/CVE-2025-32434

📥 Downloading: distilbert-base-cased-distilled-squad
   Description: QA - 261MB, balanced
   - Downloading tokenizer...


c:\Users\joems\miniconda3\envs\finrag_mlops\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\joems\.cache\huggingface\hub\models--distilbert-base-cased-distilled-squad. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


   - Downloading model weights...


...

   - Model loaded to GPU
   - Testing model...
   ✅ Success! Downloaded in 37.8s
   Test result: $2.5 billion (confidence: 0.891)

📥 Downloading: deepset/roberta-base-squad2
   Description: QA - 496MB, most accurate
   - Downloading tokenizer...


c:\Users\joems\miniconda3\envs\finrag_mlops\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\joems\.cache\huggingface\hub\models--deepset--roberta-base-squad2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


   - Downloading model weights...


Device set to use cuda:0


   - Model loaded to GPU
   - Testing model...
   ✅ Success! Downloaded in 2.1s
   Test result: $2.5 billion (confidence: 0.426)

DOWNLOAD COMPLETE - Models are cached for future use

Cache location: C:\Users\joems\.cache\huggingface
Total cache size: 27.88 GB


In [11]:
# inspect_sample_clean.py
import polars as pl
from pathlib import Path
from IPython.display import HTML, display
import textwrap

# Load data
data_path = r"D:\JoelDesktop folds_24\NEU FALL2025\MLops IE7374 Project\finrag-insights-mlops\data\exports\sec_finrag_1M_sample.parquet"
df = pl.read_parquet(data_path)

# Random sample - no seed
sample = df.sample(n=100)

def display_business_view(df, max_rows=100, wrap_length=150, table_height="700px"):
    """Display business-relevant columns with proper sentence wrapping"""
    
    # Select only business-relevant columns
    business_cols = [
        'name', 'tickers', 'section', 'report_year', 'reportDate',
        'sentence',  # Main content
        'likely_kpi', 'has_numbers', 'is_table_like', 'has_forward_looking',
        'sentenceID', 'docID', 'cik'
    ]
    
    # Filter to only columns that exist
    display_cols = [col for col in business_cols if col in df.columns]
    
    html_content = f"""
    <style>
        .sentence-table {{
            border-collapse: collapse;
            width: 100%;
            font-size: 12px;
            font-family: 'Segoe UI', Tahoma, sans-serif;
            background-color: #1e1e1e;
            color: #d4d4d4;
        }}
        .sentence-table th {{
            background-color: #2d2d30;
            color: #4EC9B0;
            padding: 10px;
            text-align: left;
            position: sticky;
            top: 0;
            z-index: 10;
            border-bottom: 2px solid #007ACC;
        }}
        .sentence-table td {{
            border: 1px solid #3a3a3a;
            padding: 8px;
            vertical-align: top;
        }}
        .sentence-table tr:hover {{
            background-color: #2a2d2e;
        }}
        .sentence-cell {{
            white-space: pre-wrap;
            word-wrap: break-word;
            max-width: 800px;
            min-width: 400px;
            line-height: 1.4;
            font-family: 'Consolas', 'Courier New', monospace;
        }}
        .table-container {{
            height: {table_height};
            overflow: auto;
            border: 1px solid #007ACC;
        }}
        .kpi-true {{
            background-color: #2a3f2a !important;
        }}
        .flag-true {{
            color: #4EC9B0;
            font-weight: bold;
        }}
        .metadata {{
            color: #808080;
            font-size: 11px;
            padding: 10px;
            background-color: #1e1e1e;
        }}
    </style>
    
    <div class="table-container">
        <table class="sentence-table">
            <thead>
                <tr>
    """
    
    # Add headers with better sizing
    for col in display_cols:
        if col == "sentence":
            width = "50%"
        elif col == "name":
            width = "10%"
        elif col in ["likely_kpi", "has_numbers"]:
            width = "5%"
        else:
            width = "auto"
        html_content += f"<th style='width: {width}'>{col}</th>"
    html_content += "</tr></thead><tbody>"
    
    # Add data rows
    for i, row in enumerate(df.head(max_rows).iter_rows(named=True)):
        kpi_class = "kpi-true" if row.get('likely_kpi', False) else ""
        html_content += f"<tr class='{kpi_class}'>"
        
        for col in display_cols:
            value = row.get(col, "")
            
            if col == "sentence":
                if isinstance(value, str):
                    # Better wrapping for readability
                    wrapped = textwrap.fill(value, width=wrap_length, 
                                          break_long_words=False,
                                          break_on_hyphens=False)
                    html_content += f"<td class='sentence-cell'>{wrapped}</td>"
                else:
                    html_content += f"<td>{value}</td>"
                    
            elif col in ["likely_kpi", "has_numbers", "is_table_like", "has_forward_looking"]:
                if value:
                    html_content += f"<td class='flag-true'>✓</td>"
                else:
                    html_content += f"<td style='color:#555'>-</td>"
                    
            elif col == "section":
                section_map = {
                    0: "Business", 1: "Risk", 7: "MD&A(old)", 
                    8: "MD&A", 9: "FinStmt", 10: "Notes", 
                    11: "Controls", 19: "Exhibits"
                }
                label = section_map.get(value, str(value))
                html_content += f"<td>{value} ({label})</td>"
                
            elif col == "tickers":
                # Handle list columns
                if isinstance(value, list):
                    html_content += f"<td>{', '.join(value)}</td>"
                else:
                    html_content += f"<td>{value}</td>"
            else:
                html_content += f"<td>{value}</td>"
        
        html_content += "</tr>"
    
    html_content += f"""
            </tbody>
        </table>
    </div>
    <div class='metadata'>
        Showing {min(max_rows, len(df))} of {len(df)} rows | 
        {len(display_cols)} columns displayed | 
        Random sample (refreshes each run)
    </div>
    """
    
    display(HTML(html_content))

# Display the sample
print(f"Sampled {len(sample)} random sentences from {len(df)} total")
print(f"Key signals: {sample['likely_kpi'].sum()} likely KPIs, {sample['has_numbers'].sum()} with numbers")
display_business_view(sample)

Sampled 100 random sentences from 1003534 total
Key signals: 26 likely KPIs, 21 with numbers


name,tickers,section,report_year,reportDate,sentence,likely_kpi,has_numbers,is_table_like,has_forward_looking,sentenceID,docID,cik
GENWORTH FINANCIAL INC,GNW,10 (Notes),2011,2011-12-31,"GENWORTH FINANCIAL, INC. NOTES TO CONSOLIDATED FINANCIAL STATEMENTS Years Ended December 31, 2011, 2010 and 2009 In June 2011, we received a subpoena from the office of the New York Attorney General relating to an industry-wide investigation of unclaimed property and escheatment practices and procedures.",-,-,-,-,0001276520_10-K_2011_section_8_1338,0001276520_10-K_2011,0001276520
EXXON MOBIL CORP,XOM,3 (3),2017,2017-12-31,The Corporation anticipates several projects will come online over the next few years providing additional production capacity.,-,-,-,✓,0000034088_10-K_2017_section_2_10,0000034088_10-K_2017,0000034088
"HANOVER INSURANCE GROUP, INC.",THG,12 (12),2017,2017-12-31,"The effectiveness of our internal control over financial reporting as of December 31, 2017 has been audited by PricewaterhouseCoopers LLP, an independent registered public accounting firm, as stated in their report which is included herein.",-,-,-,-,0000944695_10-K_2017_section_9A_14,0000944695_10-K_2017,0000944695
PNM RESOURCES INC,PNM,10 (Notes),2013,2013-12-31,"The parties agreed to a settlement of the case, which was approved by the PUCT in May 2011.",-,-,-,-,0001108426_10-K_2013_section_8_1616,0001108426_10-K_2013,0001108426
FIRST BANCORP /PR/,FBP,10 (Notes),2017,2017-12-31,"In the case of credit cards and personal lines of credit, the Corporation can cancel the unused credit facility at any time and without cause.",-,-,-,-,0001057706_10-K_2017_section_8_1494,0001057706_10-K_2017,0001057706
GENWORTH FINANCIAL INC,GNW,8 (MD&A),2019,2019-12-31,"As of December 31, 2019 and 2018, the PMIERs sufficiency ratios were in excess of $1.0 billion and $750 million, respectively, of available assets above the current and previous PMIERs requirements, respectively.",-,✓,-,-,0001276520_10-K_2019_section_7_683,0001276520_10-K_2019,0001276520
NETFLIX INC,NFLX,0 (Business),2006,2006-12-31,"In this Annual Report on Form 10-K, “Netflix,” the “Company,” “we” and the “registrant” refer to Netflix, Inc. Our investor relations Web site is located at http://ir.netflix.com.",-,-,-,-,0001065280_10-K_2006_section_1_201,0001065280_10-K_2006,0001065280
GENWORTH FINANCIAL INC,GNW,10 (Notes),2019,2019-12-31,"As of December 31, 2019 and 2018, the accumulated postretirement benefit obligation associated with these benefits was $71 million and $65 million, respectively, which we accrued in other liabilities in the consolidated balance sheets.",-,✓,-,-,0001276520_10-K_2019_section_8_709,0001276520_10-K_2019,0001276520
HAWAIIAN ELECTRIC INDUSTRIES INC,HE,10 (Notes),2018,2018-12-31,• Service Reliability Performance measured by System Average Interruption Duration and Frequency Indexes (penalties only).,-,-,-,-,0000354707_10-K_2018_section_8_658,0000354707_10-K_2018,0000354707
GOODYEAR TIRE & RUBBER CO /OH/,GT,10 (Notes),2014,2014-12-31,"16, Pension, Other Postretirement Benefits and Savings Plans.",-,-,-,✓,0000042582_10-K_2014_section_8_749,0000042582_10-K_2014,0000042582



## KPI pipeline 1 QA extraction, transformers - pipeline, Roberta, BatchProcessor, etc.

#### Move on from - RoBERTa 
1. RoBERTa is simple span extraction. 
2. FinBERT is 110MB - much smaller than RoBERTa-squad2. But critically, FinBERT is trained for sentiment classification.

In [14]:
# kpi_extraction_pipeline_v2.py - Fixed version with diagnostics
import polars as pl
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering
import re
from typing import List, Dict, Optional, Tuple
import json
from dataclasses import dataclass
from pathlib import Path
import time
from tqdm import tqdm
import logging
import warnings
warnings.filterwarnings('ignore')

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# ============= Configuration =============
@dataclass
class Config:
    """Configuration for KPI extraction pipeline"""
    data_path: str = r"D:\JoelDesktop folds_24\NEU FALL2025\MLops IE7374 Project\finrag-insights-mlops\data\exports\sec_finrag_1M_sample.parquet"
    
    model_name: str = "deepset/roberta-base-squad2"
    
    batch_size: int = 32
    confidence_threshold: float = 0.0
    device: int = 0 if torch.cuda.is_available() else -1
    max_answer_length: int = 100
    output_path: str = "extracted_kpis.parquet"
    sample_size: Optional[int] = 5000  

# ============= KPI Parser (unchanged) =============
class KPIParser:
    """Parses QA model answers into structured KPI records"""
    
    def __init__(self):
        self.patterns = {
            'currency_value': re.compile(r'\$?([\d,]+(?:\.\d+)?)\s*(?:billion|million|B|M|bn|mn)', re.I),
            'percentage': re.compile(r'([\d,]+(?:\.\d+)?)\s*%'),
            'year': re.compile(r'\b(19|20)\d{2}\b'),
            'quarter': re.compile(r'\b(Q[1-4])\s*(?:19|20)?\d{2}\b', re.I),
        }
        


    def parse_answer(self, answer_text: str, original_sentence: str, metadata: Dict) -> List[Dict]:
        """Parse QA answer into structured KPIs"""
        kpis = []
        
        for match in self.patterns['currency_value'].finditer(answer_text):
            value_str = match.group(1).replace(',', '')
            
            # FIX: Check for empty string BEFORE trying to convert to float
            if not value_str or value_str.strip() == '':
                continue
                
            try:
                value = float(value_str)
            except ValueError:
                continue  # Skip if still can't convert
            
            unit = match.group(0).split()[-1].upper()
            
            if unit in ['BILLION', 'BN', 'B']:
                multiplier = 1e9
            elif unit in ['MILLION', 'MN', 'M']:
                multiplier = 1e6
            else:
                multiplier = 1
            
            value = value * multiplier
            
            year = None
            year_matches = self.patterns['year'].findall(answer_text)
            if year_matches:
                year = year_matches[0]
            
            kpi_type = self._infer_kpi_type(answer_text, original_sentence)
            
            kpis.append({
                'kpi_type': kpi_type,
                'value': value,
                'value_raw': match.group(0),
                'year': year or metadata.get('reportDate', '')[:4] if metadata.get('reportDate') else '',
                'confidence': metadata.get('confidence', 0.5),
                'source_answer': answer_text[:200],
                'original_sentence': original_sentence[:300],
                'sentenceID': metadata.get('sentenceID', ''),
                'cik': metadata.get('cik', ''),
                'ticker': metadata.get('ticker', ''),
                'company': metadata.get('company', ''),
                'section': metadata.get('section', ''),
                'docID': metadata.get('docID', ''),
                'reportDate': metadata.get('reportDate', '')  # ADD THIS for business context
            })
        
        return kpis



    def _infer_kpi_type(self, answer_text: str, original_sentence: str) -> str:
        """Infer KPI type from context"""
        context = (answer_text + " " + original_sentence).lower()
        
        if any(term in context for term in ['revenue', 'sales', 'turnover']):
            return 'revenue'
        elif any(term in context for term in ['income', 'earnings', 'profit']):
            return 'net_income' if 'net' in context else 'operating_income' if 'operating' in context else 'income'
        elif 'ebitda' in context:
            return 'ebitda'
        elif 'margin' in context:
            return 'gross_margin' if 'gross' in context else 'operating_margin' if 'operating' in context else 'margin'
        elif any(term in context for term in ['eps', 'per share']):
            return 'eps'
        else:
            return 'financial_metric'




# ============= Batch Processor with Better Logging =============
class BatchProcessor:
    """Handles efficient batch processing for GPU"""
    
    def __init__(self, qa_pipeline, batch_size: int = 8, confidence_threshold: float = 0.0):
        self.qa_pipeline = qa_pipeline
        self.batch_size = batch_size
        self.confidence_threshold = confidence_threshold
        self.parser = KPIParser()
        logger.info(f"BatchProcessor initialized with batch_size={batch_size}")
    
    def process_batch(self, sentences: List[str], metadata_list: List[Dict]) -> List[Dict]:
        """Process a batch of sentences through QA pipeline"""
        all_kpis = []
        
        # questions for speed; increase later
        questions = [
            "What are the financial metrics, values, and years mentioned?",
            "What revenue or earnings figures are reported?"
        ]
        
        logger.debug(f"Processing batch of {len(sentences)} sentences")
        
        for q_idx, question in enumerate(questions):
            try:
                qa_inputs = [
                    {"question": question, "context": sentence}
                    for sentence in sentences
                ]
                
                # TIME THE PIPELINE CALL
                start = time.time()
                answers = self.qa_pipeline(qa_inputs)
                logger.debug(f"Question {q_idx+1} processed in {time.time()-start:.2f}s")

                ## during exploration, capture everything and analyze the confidence distribution later ??
                for answer, sentence, metadata in zip(answers, sentences, metadata_list):
                    if answer['score'] >= Config.confidence_threshold:
                        metadata['confidence'] = answer['score']
                        kpis = self.parser.parse_answer(
                            answer['answer'],
                            sentence,
                            metadata
                        )
                        all_kpis.extend(kpis)
            
            except Exception as e:
                logger.error(f"Batch processing error: {e}")
                continue
        
        return all_kpis

# ============= Main KPI Extractor with Diagnostics =============
class KPIExtractor:
    """Main orchestrator for KPI extraction pipeline"""
    
    def __init__(self, config: Config):
        self.config = config
        
        # BETTER DEVICE LOGGING
        if torch.cuda.is_available():
            logger.info(f"CUDA is available")
            logger.info(f"GPU Device: {torch.cuda.get_device_name(0)}")
            logger.info(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
        else:
            logger.warning("CUDA not available, using CPU")
        
        logger.info(f"Loading model: {config.model_name}")
        
        # TIME MODEL LOADING
        start = time.time()
        try:
            self.qa_pipeline = pipeline(
                "question-answering",
                model=config.model_name,
                device=config.device,
                max_answer_len=config.max_answer_length
            )
            logger.info(f"Model loaded successfully in {time.time()-start:.1f}s")
        except Exception as e:
            logger.error(f"Failed to load model: {e}")
            raise

        self.batch_processor = BatchProcessor(self.qa_pipeline, config.batch_size, config.confidence_threshold)

        ## self.numeric_pattern = re.compile(r'\d+\.?\d*[BMK%]?|\$\d+')
        
        # TEST THE PIPELINE
        logger.info("Testing pipeline with sample...")
        test_result = self.qa_pipeline(
            question="What is the revenue?", 
            context="The revenue was $100 million in 2020"
        )
        logger.info(f"Test successful: {test_result}")
    


    def extract_from_dataset(self, show_diagnostics: bool = True, diagnostic_interval: int = 200) -> pl.DataFrame:
        """Main extraction pipeline - NO PRE-FILTERING
        
        Args:
            show_diagnostics: Whether to show progress examples during processing
            diagnostic_interval: Show example every N sentences (only if show_diagnostics=True)
        """
        logger.info(f"Loading data from {self.config.data_path}")
        
        # Load data
        df_lazy = pl.scan_parquet(self.config.data_path)
        
        if self.config.sample_size:
            logger.info(f"Random sampling {self.config.sample_size} rows")
            df_full = df_lazy.collect()
            df = df_full.sample(n=min(self.config.sample_size, len(df_full)), seed=42)
        else:
            df = df_lazy.collect()
        
        # Select needed columns
        start = time.time()
        df = df.select([
            "sentence", "tickers", "name", "section", 
            "reportDate", "cik", "sentenceID", "docID"
        ])
        logger.info(f"Data loaded in {time.time()-start:.1f}s")
        logger.info(f"Processing {len(df)} sentences")
        
        # Show data distribution only if diagnostics enabled
        if show_diagnostics:
            section_counts = df.group_by("section").count().sort("count", descending=True)
            logger.info("Section distribution in sample:")
            for row in section_counts.head(5).iter_rows(named=True):
                logger.info(f"  Section {row['section']}: {row['count']} sentences")
        
        # Process everything
        all_kpis = []
        sentences = df["sentence"].to_list()
        
        # Metadata mapping
        metadata_list = []
        for row in df.iter_rows(named=True):
            metadata_list.append({
                'ticker': row['tickers'],
                'company': row['name'],
                'section': row['section'],
                'reportDate': str(row['reportDate']) if row['reportDate'] else '',
                'cik': row['cik'],
                'sentenceID': row['sentenceID'],
                'docID': row['docID']
            })
        
        # Process batches
        for i in tqdm(range(0, len(sentences), self.config.batch_size), desc="Processing batches"):
            batch_sentences = sentences[i:i + self.config.batch_size]
            batch_metadata = metadata_list[i:i + self.config.batch_size]
            
            kpis = self.batch_processor.process_batch(
                batch_sentences,
                batch_metadata
            )
            all_kpis.extend(kpis)
            
            # Diagnostic output only if enabled
            if show_diagnostics and i > 0 and i % diagnostic_interval == 0:
                logger.info(f"\n--- Progress at sentence {i}/{len(sentences)} ---")
                logger.info(f"KPIs found so far: {len(all_kpis)}")
                if all_kpis:
                    latest = all_kpis[-1]
                    logger.info(f"Latest KPI: {latest['kpi_type']} = {latest['value_raw']}")
                    logger.info(f"From: {latest['original_sentence'][:100]}...")
                    logger.info(f"Confidence: {latest['confidence']:.3f}")
        
        logger.info(f"\nExtraction complete. Found {len(all_kpis)} total KPI mentions")
        
        if all_kpis:
            kpi_df = pl.DataFrame(all_kpis)
            kpi_df = kpi_df.unique(subset=['kpi_type', 'value', 'year', 'ticker', 'sentenceID'])
            logger.info(f"After deduplication: {len(kpi_df)} unique KPIs")
            return kpi_df
        else:
            return pl.DataFrame()
        



# ============= Quick Test Function =============
def quick_test():
    """Quick test with minimal data"""
    config = Config()
    config.sample_size = 1000  
    config.batch_size = 16     
    
    logger.info("="*60)
    logger.info("RUNNING QUICK TEST")
    logger.info("="*60)
    
    extractor = KPIExtractor(config)
    
    start_time = time.time()
    kpi_df = extractor.extract_from_dataset(show_diagnostics=True, diagnostic_interval=200)
    elapsed = time.time() - start_time
    
    if kpi_df is not None and len(kpi_df) > 0:
        logger.info(f"\nResults Summary:")
        logger.info(f"- Extracted {len(kpi_df)} KPIs in {elapsed:.1f}s")
        logger.info(f"- Columns: {kpi_df.columns}")
        
        # Enhanced business view - just select more columns we already have
        if 'sentenceID' in kpi_df.columns:
            logger.info("✓ sentenceID preserved for accuracy testing")
            
            # Better sample with business context
            business_sample = kpi_df.select([
                'company', 'ticker', 'kpi_type', 'value', 
                'year', 'section', 'reportDate', 'confidence'
            ]).head(10)
            
            print("\nBusiness View - Top 10 KPIs:")
            print(business_sample)
            
            # Group by KPI type to show distribution
            kpi_distribution = kpi_df.group_by('kpi_type').count().sort('count', descending=True)
            print("\nKPI Type Distribution:")
            print(kpi_distribution)
            
            # Show confidence stats
            print(f"\nConfidence Stats:")
            print(f"  Mean: {kpi_df['confidence'].mean():.3f}")
            print(f"  Min: {kpi_df['confidence'].min():.3f}")
            print(f"  Max: {kpi_df['confidence'].max():.3f}")
            
            # Save with all columns -- some errors around sink csv issue.
            # kpi_df.write_csv("extracted_kpis_test.csv")
            logger.info(f"Saved full results to: extracted_kpis_test.csv")
        
        return kpi_df
    else:
        logger.warning("No KPIs extracted")
        return pl.DataFrame()
    



# Run the test
if __name__ == "__main__":
    kpi_results = quick_test()

2025-10-21 05:01:21,943 - INFO - ============================================================
2025-10-21 05:01:21,944 - INFO - RUNNING QUICK TEST
2025-10-21 05:01:21,945 - INFO - ============================================================
2025-10-21 05:01:21,946 - INFO - CUDA is available
2025-10-21 05:01:21,948 - INFO - GPU Device: NVIDIA GeForce RTX 3080 Ti Laptop GPU
2025-10-21 05:01:21,949 - INFO - GPU Memory: 17.18 GB
2025-10-21 05:01:21,949 - INFO - Loading model: deepset/roberta-base-squad2
Device set to use cuda:0
2025-10-21 05:01:24,292 - INFO - Model loaded successfully in 2.3s
2025-10-21 05:01:24,293 - INFO - BatchProcessor initialized with batch_size=16
2025-10-21 05:01:24,294 - INFO - Testing pipeline with sample...
2025-10-21 05:01:24,315 - INFO - Test successful: {'score': 0.0006287800497375429, 'start': 16, 'end': 28, 'answer': '$100 million'}
2025-10-21 05:01:24,316 - INFO - Loading data from D:\JoelDesktop folds_24\NEU FALL2025\MLops IE7374 Project\finrag-insights-ml


Business View - Top 10 KPIs:
shape: (10, 8)
┌────────────────┬───────────┬───────────────┬──────────┬──────┬─────────┬────────────┬────────────┐
│ company        ┆ ticker    ┆ kpi_type      ┆ value    ┆ year ┆ section ┆ reportDate ┆ confidence │
│ ---            ┆ ---       ┆ ---           ┆ ---      ┆ ---  ┆ ---     ┆ ---        ┆ ---        │
│ str            ┆ list[str] ┆ str           ┆ f64      ┆ str  ┆ i64     ┆ str        ┆ f64        │
╞════════════════╪═══════════╪═══════════════╪══════════╪══════╪═════════╪════════════╪════════════╡
│ GENWORTH       ┆ ["GNW"]   ┆ financial_met ┆ 2.76e8   ┆ 2011 ┆ 8       ┆ 2011-12-31 ┆ 0.000004   │
│ FINANCIAL INC  ┆           ┆ ric           ┆          ┆      ┆         ┆            ┆            │
│ PNM RESOURCES  ┆ ["PNM"]   ┆ financial_met ┆ 8.2000e6 ┆ 20   ┆ 10      ┆ 2013-12-31 ┆ 0.000807   │
│ INC            ┆           ┆ ric           ┆          ┆      ┆         ┆            ┆            │
│ QUALCOMM       ┆ ["QCOM"]  ┆ financial_met ┆

#### yes—
- on KPI-style extraction from 10-K/10-Q text, modern LLMs (e.g., Qwen2.5, DeepSeek-V3, Mixtral) beat BERT-class models like FinBERT/ProBERT on entity/value/date/currency labeling and structured JSON extraction.
- newer LLMs do multi-entity span reasoning + format-following much better
- MoE/large dense models generalize across wildly varied KPI phrasings


#### previous work:
```
batch = ["sentence1", "sentence2", "sentence3", ..., "sentence16"]
results = qa_pipeline(batch)  # All 16 processed in parallel on GPU
```
- Transformers library and PyTorch handle this parallelization automatically.
- Sad, Qwen-14B fills most of 16GB VRAM, Ollama runs as a REST server handling one request at a time.
- Each generation needs additional memory for KV cache

### KPI pipeline 2 attempt: Tried 14b param model. V slow.
#### ----------------------------------------------------------------------------------------------------------------------

In [20]:
import polars as pl
import json
import time
from abc import ABC, abstractmethod
from dataclasses import dataclass
from typing import Dict, List, Optional, Any, Union
from pathlib import Path
from tqdm import tqdm
import logging
from datetime import datetime

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ============= Configuration =============
@dataclass
class ExtractionConfig:
    """Configuration for KPI extraction pipeline"""
    data_path: str
    output_dir: str = "./kpi_outputs"
    sample_size: Optional[int] = None  # None = full dataset
    random_seed: int = 42
    checkpoint_interval: int = 1000
    backend: str = "ollama"  # ollama, transformers, anthropic, openai
    model_name: str = "qwen2.5:14b-instruct-q4_K_M"
    temperature: float = 0.0
    max_retries: int = 2
    
    def __post_init__(self):
        Path(self.output_dir).mkdir(parents=True, exist_ok=True)

# ============= Abstract Base Extractor =============
class BaseKPIExtractor(ABC):
    """Abstract base class for different extraction backends"""
    
    @abstractmethod
    def extract(self, text: str, metadata: Dict[str, Any]) -> List[Dict]:
        """Extract KPIs from text"""
        pass
    
    @abstractmethod
    def health_check(self) -> bool:
        """Check if backend is available"""
        pass
    
    def get_prompt(self, text: str, metadata: Dict[str, Any]) -> str:
        """Common prompt template across backends"""
        return f"""Extract ALL financial KPIs from this text. Return ONLY valid JSON array.

Text: {text}
Company: {metadata.get('company', 'Unknown')}
Section: {metadata.get('section', '')}
Report Date: {metadata.get('reportDate', '')}

For each KPI found, include:
- metric_type: revenue|earnings|margin|debt|cash_flow|capex|operational|other
- value: numeric value (no commas)
- unit: millions|billions|percent|ratio|count
- year: YYYY or null
- quarter: Q1|Q2|Q3|Q4 or null
- confidence: 0.0-1.0
- explanation: brief context

Example: [{{"metric_type":"revenue","value":2500,"unit":"millions","year":2023,"confidence":0.9,"explanation":"annual revenue"}}]

If no KPIs found, return: []
"""

# ============= Ollama Implementation =============
class OllamaExtractor(BaseKPIExtractor):
    """Local Ollama-based extraction"""
    
    def __init__(self, model_name: str, temperature: float = 0.0):
        import requests
        self.model = model_name
        self.temperature = temperature
        self.base_url = "http://localhost:11434"
        self.requests = requests
        
    def health_check(self) -> bool:
        try:
            resp = self.requests.get(f"{self.base_url}/api/tags")
            models = [m['name'] for m in resp.json().get('models', [])]
            if self.model not in models:
                logger.warning(f"Model {self.model} not found. Available: {models}")
                return False
            return True
        except Exception as e:
            logger.error(f"Ollama not available: {e}")
            return False
    
    def extract(self, text: str, metadata: Dict[str, Any]) -> List[Dict]:
        prompt = self.get_prompt(text, metadata)
        
        payload = {
            "model": self.model,
            "prompt": prompt,
            "stream": False,
            "temperature": self.temperature,
            "format": "json"
        }
        
        try:
            resp = self.requests.post(
                f"{self.base_url}/api/generate",
                json=payload,
                timeout=30
            )
            
            if resp.status_code == 200:
                result = json.loads(resp.json()['response'])
                if isinstance(result, list):
                    # Add metadata to each KPI
                    for kpi in result:
                        kpi.update({
                            'sentenceID': metadata.get('sentenceID'),
                            'cik': metadata.get('cik'),
                            'ticker': metadata.get('ticker')
                        })
                    return result
                return []
        except Exception as e:
            logger.debug(f"Extraction failed: {e}")
            return []

# ============= Cloud API Implementation =============
class CloudAPIExtractor(BaseKPIExtractor):
    """Cloud provider API extraction (Anthropic, OpenAI, etc.)"""
    
    def __init__(self, provider: str, api_key: str, model_name: str, temperature: float = 0.0):
        self.provider = provider
        self.api_key = api_key
        self.model = model_name
        self.temperature = temperature
        
        if provider == "anthropic":
            from anthropic import Anthropic
            self.client = Anthropic(api_key=api_key)
        elif provider == "openai":
            from openai import OpenAI
            self.client = OpenAI(api_key=api_key)
        else:
            raise ValueError(f"Unknown provider: {provider}")
    
    def health_check(self) -> bool:
        # Simple check - could be enhanced
        return self.api_key is not None and len(self.api_key) > 0
    
    def extract(self, text: str, metadata: Dict[str, Any]) -> List[Dict]:
        prompt = self.get_prompt(text, metadata)
        
        try:
            if self.provider == "anthropic":
                response = self.client.messages.create(
                    model=self.model,
                    max_tokens=1000,
                    temperature=self.temperature,
                    messages=[{"role": "user", "content": prompt}]
                )
                result = json.loads(response.content[0].text)
            
            elif self.provider == "openai":
                response = self.client.chat.completions.create(
                    model=self.model,
                    temperature=self.temperature,
                    response_format={"type": "json_object"},
                    messages=[{"role": "user", "content": prompt}]
                )
                result = json.loads(response.choices[0].message.content)
            
            if isinstance(result, list):
                for kpi in result:
                    kpi.update({
                        'sentenceID': metadata.get('sentenceID'),
                        'cik': metadata.get('cik'),
                        'ticker': metadata.get('ticker')
                    })
                return result
            return []
            
        except Exception as e:
            logger.debug(f"Cloud API extraction failed: {e}")
            return []

# ============= Transformers Fallback =============
class TransformersExtractor(BaseKPIExtractor):
    """Fallback to smaller transformer models"""
    
    def __init__(self, model_name: str = "deepset/roberta-base-squad2"):
        from transformers import pipeline
        import torch
        
        self.device = 0 if torch.cuda.is_available() else -1
        self.qa_pipeline = pipeline(
            "question-answering",
            model=model_name,
            device=self.device
        )
    
    def health_check(self) -> bool:
        try:
            test = self.qa_pipeline(
                question="What is the value?",
                context="The value is $100 million"
            )
            return True
        except:
            return False
    
    def extract(self, text: str, metadata: Dict[str, Any]) -> List[Dict]:
        """Simplified extraction for QA models"""
        questions = [
            "What are the financial metrics and values?",
            "What is the revenue or earnings?"
        ]
        
        kpis = []
        for question in questions:
            try:
                answer = self.qa_pipeline(question=question, context=text)
                if answer['score'] > 0.3:
                    # Basic parsing - less sophisticated than LLM
                    kpi = {
                        'metric_type': 'financial_metric',
                        'value': answer['answer'],
                        'confidence': answer['score'],
                        'sentenceID': metadata.get('sentenceID'),
                        'cik': metadata.get('cik'),
                        'ticker': metadata.get('ticker')
                    }
                    kpis.append(kpi)
            except:
                continue
        
        return kpis

# ============= Data Preparation =============
class FinancialDataset:
    """Dataset preparation with proper sampling"""
    
    def __init__(self, data_path: str, sample_size: Optional[int] = None, 
                 random_seed: int = 42):
        self.data_path = data_path
        self.sample_size = sample_size
        self.random_seed = random_seed
        self.df = None
        
    def prepare(self) -> pl.DataFrame:
        """Load and prepare dataset with proper sampling"""
        logger.info(f"Loading data from {self.data_path}")
        
        # Lazy load for efficiency
        df = pl.scan_parquet(self.data_path).collect()
        
        # Apply sampling if specified
        if self.sample_size and self.sample_size < len(df):
            logger.info(f"Random sampling {self.sample_size} from {len(df)} rows")
            df = df.sample(n=self.sample_size, seed=self.random_seed)
        
        logger.info(f"Dataset ready: {len(df)} sentences")
        
        # Show distribution
        section_dist = df.group_by("section").count().sort("count", descending=True)
        logger.info("Section distribution:")
        for row in section_dist.head(5).iter_rows(named=True):
            logger.info(f"  Section {row['section']}: {row['count']}")
        
        self.df = df
        return df

# ============= Main Pipeline =============
class FinancialKPIPipeline:
    """Main extraction pipeline with checkpointing"""
    
    def __init__(self, config: ExtractionConfig):
        self.config = config
        self.extractor = self._initialize_extractor()
        self.dataset = FinancialDataset(
            config.data_path,
            config.sample_size,
            config.random_seed
        )
        self.checkpoint_path = Path(config.output_dir) / "checkpoint.json"
        
    def _initialize_extractor(self) -> BaseKPIExtractor:
        """Initialize appropriate backend"""
        if self.config.backend == "ollama":
            extractor = OllamaExtractor(
                self.config.model_name,
                self.config.temperature
            )
        elif self.config.backend == "anthropic":
            import os
            extractor = CloudAPIExtractor(
                provider="anthropic",
                api_key=os.getenv("ANTHROPIC_API_KEY"),
                model_name=self.config.model_name,
                temperature=self.config.temperature
            )
        elif self.config.backend == "transformers":
            extractor = TransformersExtractor(self.config.model_name)
        else:
            raise ValueError(f"Unknown backend: {self.config.backend}")
        
        if not extractor.health_check():
            raise RuntimeError(f"Backend {self.config.backend} not available")
        
        logger.info(f"Initialized {self.config.backend} backend")
        return extractor
    
    def _load_checkpoint(self) -> int:
        """Load checkpoint if exists"""
        if self.checkpoint_path.exists():
            with open(self.checkpoint_path, 'r') as f:
                checkpoint = json.load(f)
                logger.info(f"Resuming from sentence {checkpoint['last_processed']}")
                return checkpoint['last_processed']
        return 0
    
    def _save_checkpoint(self, last_idx: int, kpis: List[Dict]):
        """Save progress checkpoint"""
        checkpoint = {
            'last_processed': last_idx,
            'timestamp': datetime.now().isoformat(),
            'kpis_found': len(kpis)
        }
        with open(self.checkpoint_path, 'w') as f:
            json.dump(checkpoint, f)
    
    def run(self) -> pl.DataFrame:
        """Execute extraction pipeline"""
        # Prepare data
        df = self.dataset.prepare()
        
        # Check for existing checkpoint
        start_idx = self._load_checkpoint()
        
        # Process sentences
        all_kpis = []
        
        for idx, row in enumerate(tqdm(
            df.iter_rows(named=True), 
            total=len(df),
            initial=start_idx,
            desc="Extracting KPIs"
        )):
            if idx < start_idx:
                continue
            
            metadata = {
                'sentenceID': row['sentenceID'],
                'cik': row['cik'],
                'ticker': row.get('tickers', row.get('ticker')),
                'company': row.get('name', row.get('company')),
                'section': row['section'],
                'reportDate': str(row.get('reportDate', ''))
            }
            
            # Extract with retries
            for attempt in range(self.config.max_retries):
                kpis = self.extractor.extract(row['sentence'], metadata)
                if kpis:
                    all_kpis.extend(kpis)
                    break
                time.sleep(1)  # Brief pause between retries
            
            # Checkpoint periodically
            if idx > 0 and idx % self.config.checkpoint_interval == 0:
                self._save_checkpoint(idx, all_kpis)
                logger.info(f"Checkpoint: {idx}/{len(df)}, KPIs: {len(all_kpis)}")
        
        # Create final dataframe
        if all_kpis:
            kpi_df = pl.DataFrame(all_kpis)
            output_path = Path(self.config.output_dir) / f"kpis_{datetime.now():%Y%m%d_%H%M%S}.parquet"
            kpi_df.write_parquet(output_path)
            logger.info(f"Saved {len(kpi_df)} KPIs to {output_path}")
            return kpi_df
        else:
            logger.warning("No KPIs extracted")
            return pl.DataFrame()

# ============= Usage =============


## Allows for ollama, anthropic, transformers backends.
if __name__ == "__main__":
    # Configure extraction
    config = ExtractionConfig(

        # data_path=r"D:\path\to\sec_finrag_1M_sample.parquet",
        data_path=r"D:\JoelDesktop folds_24\NEU FALL2025\MLops IE7374 Project\finrag-insights-mlops\data\exports\sec_finrag_1M_sample.parquet",
        sample_size=100, 
        backend="ollama",  # or "anthropic", "transformers"
        model_name="qwen2.5:14b-instruct-q4_K_M",
        checkpoint_interval=100
    )
    
    # Run pipeline
    pipeline = FinancialKPIPipeline(config)
    kpi_df = pipeline.run()
    
    # Display results
    if len(kpi_df) > 0:
        print(f"\nExtracted {len(kpi_df)} KPIs")
        print("\nSample results:")
        print(kpi_df.select(['metric_type', 'value', 'unit', 'confidence']).head(10))

2025-10-21 06:01:12,192 - INFO - Initialized ollama backend
2025-10-21 06:01:12,193 - INFO - Loading data from D:\JoelDesktop folds_24\NEU FALL2025\MLops IE7374 Project\finrag-insights-mlops\data\exports\sec_finrag_1M_sample.parquet
2025-10-21 06:01:12,538 - INFO - Random sampling 100 from 1003534 rows
2025-10-21 06:01:12,616 - INFO - Dataset ready: 100 sentences
2025-10-21 06:01:12,616 - INFO - Section distribution:
2025-10-21 06:01:12,624 - INFO -   Section 10: 37
2025-10-21 06:01:12,624 - INFO -   Section 1: 25
2025-10-21 06:01:12,624 - INFO -   Section 8: 15
2025-10-21 06:01:12,624 - INFO -   Section 19: 14
2025-10-21 06:01:12,624 - INFO -   Section 0: 4
Extracting KPIs: 100%|██████████| 100/100 [14:36<00:00,  8.77s/it]
2025-10-21 06:15:49,311 - WARNING - No KPIs extracted


#### ----------------------------------------------------------------------------------------------------------------------

### Improvement: llama CPP server + GGUF model + HTTP calls from Jupyter notebook.
- Pretty easy—download one EXE, a GGUF model, run a command, and call it from Jupyter notebook like any REST API. No WSL/Linux needed..
- Download → double-click/run → call a REST endpoint. 
- On Windows + CUDA, llama.cpp server with --cont-batching gives you real, batched throughput. Swap Ollama call for the HTTP call above and you’ll see a big step up—especially with 7B (or 3B) GGUF.

- PyTorch in Conda ships its own CUDA/cuDNN runtime. It’s used only by PyTorch.
- llama.cpp (cuBLAS build) is a separate program. It uses CUDA via NVIDIA driver (and the CUDA runtime it’s linked against). It does not depend on Conda CUDA at all.
- Because you call llama.cpp via HTTP, Python kernel/env is irrelevant to GPU acceleration—think “call a local web service.”


```
cd C:\llama
.\llama-server.exe `
  -m "C:\models\qwen2.5-7b\Qwen2.5-7B-Instruct-Q4_K_M.gguf" `
  --port 8080 `
  --ctx-size 1024 `
  --gpu-layers -1 `
  --cont-batching `
  --parallel 16 `
  --api

--gpu-layers -1 uses 3080 Ti fully.
--cont-batching gives real server-side concurrency.
--parallel controls how many simultaneous requests it merges.
```

- nvidia-smi = NVIDIA driver info + the maximum CUDA version the driver supports (runtime capability). CUDA 13.0 means the driver can run binaries built for up to CUDA 13 (and earlier 12.x).
- nvcc --version = your installed CUDA Toolkit compiler version (used for building CUDA code), here 12.1.


### Details 1:
- you’re just running a tiny web service on your own PC that loads a GGUF model into GPU VRAM and answers HTTP requests. “chat/completions” bit is only the API shape.
- llama.cpp : fast C/CUDA program that runs LLMs locally.
- llama-server.exe : precompiled Windows executable of llama.cpp with REST API support.
- Loads a model once into VRAM and exposes a HTTP API on http://localhost:PORT.
- GGUF model: contains the LLM weights in a quantized format optimized for llama.cpp.
- download it once and point the server to it with -m <path.gguf>.
- CUDA/cuBLAS DLLs: use the NVIDIA driver, not your Conda/PyTorch CUDA. Let llama.cpp run on your NVIDIA GPU.

### Details 2:
- llama-server purposely mimics the OpenAI Chat Completions API. 
- i.e. can reuse tons of existing client code/tools., JSON output controls like response_format {}, 
- convention for message formatting (system/user/assistant) and options (max_tokens, temperature, etc.), even though it’s all local.
- Two ways to run models with llama.cpp: llama-cli or llama.exe runs a single prompt → single output in the terminal, llama-server.exe loads the model once, then you send many requests from Python. 


```
Your Notebook  --HTTP POST-->  llama-server (on your PC)  --CUDA-->  GPU
                                 ^  keeps model in VRAM  ^
                                 |  merges requests      |
                               OpenAI-style JSON        GGUF weights
```

### Details 3:
**in the other hf_llama_debug.ipynb file**, just downloaded - ✓ downloaded: C:\llama_server\models\qwen2p5_3b\Qwen2.5-3B-Instruct-Q4_K_M.gguf. 

**Server life cycle**:
1. You launch llama-server.exe -m <model.gguf> --api --cont-batching ...
2. It loads weights into VRAM once (watch with nvidia-smi).
3. It listens on localhost:8080.

**Request life cycle**:
1. Python builds a short prompt with your schema.
2. POST /v1/chat/completions (JSON options like max_tokens, temperature).
3. Server decodes on GPU (possibly batched with other requests).
4. You receive JSON (your KPI objects), parse, validate, save.

**Scaling knobs**
1. Smaller model (3B) → faster.
2. --parallel up to ~8–24 → more concurrent requests merged.
3. Short prompts + max_tokens<=128 → big speed gains.
4. Keep temperature=0 and top_p low for deterministic, compact outputs.

### Trace one sentence through the pipeline:
**Input**: "Operating income increased 23% to $1.4 billion, driven by cloud revenue growth"
1, 2, 3, 4. - Now deleted. Classification - NER - Dependency Parsing - Relation Linking etc. Not needed.

5. Improvement: 
- Count temporal markers (years, quarters, months), Count value entities (numbers with units), Count KPI keywords ?
- because sentences with multiple temporal markers + multiple values.=
- i.e. anchor points, metric clusters, binding.
- **No** - Cut down on the 'temporal matrices of data' concept.
- Transformers know PEs, Attn mechanisms, Context windows, Fin Pretraining.
- Dont go for - Dependency parsing, Custom NER training, Grammar rules, Temporal matrices.
- Go for - FinBERT 


In [10]:
## Testing 1: Probe eps.

import subprocess, requests, time, os

LLAMA_DIR = r"C:\llama_server\llama_cpp_b6814"   # adjust if different
exe = os.path.join(LLAMA_DIR, "llama-server.exe")

# 1️⃣  start server with --api and --props (no model, short run)
proc = subprocess.Popen(
    [exe, "--api", "--props", "--port", "8080"],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
)
time.sleep(5)  # give it a few seconds to boot

# 2️⃣  probe endpoints
for url in [
    "http://localhost:8080/health",
    "http://localhost:8080/v1/models",
    "http://localhost:8080/props",
    "http://localhost:8080/routes",
]:
    try:
        r = requests.get(url, timeout=1)
        print(f"{url:<45} -> {r.status_code}")
    except Exception as e:
        print(f"{url:<45} -> FAIL ({e})")

# 3️⃣  clean up
proc.terminate()



http://localhost:8080/health                  -> 200
http://localhost:8080/v1/models               -> 200
http://localhost:8080/props                   -> 200
http://localhost:8080/routes                  -> 404


### Llamma server cpp fails massively, even at the setup stage. Issues, hard to track argument conflicts, server dying out for no reason, not starting or loading.

### Choosing llama cpp python.



In [5]:
from llama_cpp import Llama
import time

MODEL_PATH = r"C:\llama_server\models\qwen2p5_7b\Qwen2.5-7B-Instruct-Q5_K_M.gguf"

print("Loading model...")
start = time.time()

llm = Llama(
    model_path=MODEL_PATH,
    n_gpu_layers=35,  # Use GPU
    n_ctx=4096,
    verbose=True  # See if CUDA is being used
)

print(f"✓ Loaded in {time.time()-start:.1f}s")
print(f"✓ CUDA available: {llm.model_params.n_gpu_layers > 0}")


llama_model_loader: loaded meta data with 38 key-value pairs and 339 tensors from C:\llama_server\models\qwen2p5_7b\Qwen2.5-7B-Instruct-Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Qwen2.5 7B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Qwen2.5
llama_model_loader: - kv   5:                         general.size_label str              = 7B
llama_model_loader: - kv   6:                            general.license str              = apache-2.0
llama_model_loader: 

Loading model...


init_tokenizer: initializing tokenizer for type 2
load: control token: 151661 '<|fim_suffix|>' is not marked as EOG
load: control token: 151649 '<|box_end|>' is not marked as EOG
load: control token: 151647 '<|object_ref_end|>' is not marked as EOG
load: control token: 151654 '<|vision_pad|>' is not marked as EOG
load: control token: 151659 '<|fim_prefix|>' is not marked as EOG
load: control token: 151648 '<|box_start|>' is not marked as EOG
load: control token: 151644 '<|im_start|>' is not marked as EOG
load: control token: 151646 '<|object_ref_start|>' is not marked as EOG
load: control token: 151650 '<|quad_start|>' is not marked as EOG
load: control token: 151651 '<|quad_end|>' is not marked as EOG
load: control token: 151652 '<|vision_start|>' is not marked as EOG
load: control token: 151653 '<|vision_end|>' is not marked as EOG
load: control token: 151655 '<|image_pad|>' is not marked as EOG
load: control token: 151656 '<|video_pad|>' is not marked as EOG
load: control token: 151

✓ Loaded in 1.0s
✓ CUDA available: True


CPU : SSE3 = 1 | SSSE3 = 1 | AVX = 1 | AVX2 = 1 | F16C = 1 | FMA = 1 | LLAMAFILE = 1 | OPENMP = 1 | REPACK = 1 | 
Model metadata: {'general.name': 'Qwen2.5 7B Instruct', 'general.architecture': 'qwen2', 'general.type': 'model', 'general.basename': 'Qwen2.5', 'general.finetune': 'Instruct', 'general.size_label': '7B', 'general.license': 'apache-2.0', 'qwen2.attention.head_count_kv': '4', 'general.license.link': 'https://huggingface.co/Qwen/Qwen2.5-7B-Instruct/blob/main/LICENSE', 'general.base_model.count': '1', 'general.base_model.0.name': 'Qwen2.5 7B', 'general.base_model.0.organization': 'Qwen', 'general.base_model.0.repo_url': 'https://huggingface.co/Qwen/Qwen2.5-7B', 'qwen2.block_count': '28', 'qwen2.context_length': '32768', 'quantize.imatrix.dataset': '/training_dir/calibration_datav3.txt', 'qwen2.embedding_length': '3584', 'general.quantization_version': '2', 'tokenizer.ggml.bos_token_id': '151643', 'qwen2.feed_forward_length': '18944', 'qwen2.attention.head_count': '28', 'tokeni

In [ ]:
# === Local LLM KPI extractor (llama.cpp server, Windows, CUDA) ===
# lean version: download GGUF (if missing) -> start server -> run 7-sentence test.

# native caused TONS of errors. server build that supports the OpenAI-style /v1/* API (and --props).
# 
# --------------------------------------------------------------------------------------------------------------------


# test_sentences = [
#     "For 2018, we estimate that mobile advertising revenue represented approximately 92% of total advertising revenue, as compared with approximately 88% in 2017.",
#     "Realization of deferred tax assets associated with net operating loss and credit carryforwards is dependent upon generating sufficient taxable income prior to their expiration in the appropriate tax jurisdiction.",
#     "A reduction of 85 million BOE was recorded in Canada, primarily from commodity price effects at Kaybob Duvernay.",
#     "2018 compared to 2017 The Other segment reported net income of $14 million for the year ended December 31, 2018, compared to net loss of $71 million for the year ended December 31, 2017.",
#     "Future policy benefits for individual life insurance and annuity policies consider crediting rates ranging from 2 1/2% to 6% for life insurance and 2% to 9 1/2% for annuities.",
#     "The following table presents restructuring activity for the years ended June 30, 2019 and 2018: Separation Costs Employee separation charges for the years ended June 30, 2019 and 2018 relate to severance packages for approximately 1,810 and 2,720 employees, respectively.",
#     "In May 2017, we issued $750.0 million of 2.35 percent fixed-rate notes due in May 2022, $750.0 million of 3.10 percent fixed-rate notes due in May 2027, and $750.0 million of 3.95 percent fixed-rate notes due in May 2047, with interest to be paid semi-annually."
#     ]



In [6]:
"""
SIMPLE KPI Extractor using llama-cpp-python directly
No server, no async, no complexity - just works!
"""

from llama_cpp import Llama
import json
import re
from typing import List, Dict
import time

# =============================================================================
# STEP 1: Install llama-cpp-python with CUDA support
# =============================================================================
"""
Run this in your terminal ONCE:

pip uninstall llama-cpp-python -y
set CMAKE_ARGS=-DGGML_CUDA=on
pip install llama-cpp-python --no-cache-dir

That's it for setup!
"""

# =============================================================================
# CONFIGURATION
# =============================================================================
MODEL_PATH = r"C:\llama_server\models\qwen2p5_7b\Qwen2.5-7B-Instruct-Q5_K_M.gguf"

# Model parameters
N_GPU_LAYERS = 35  # Adjust based on your VRAM (35 should work for 7B model)
N_CTX = 4096       # Context window
N_BATCH = 512      # Batch size for prompt processing
TEMPERATURE = 0.1   # Low for consistent output

# =============================================================================
# LOAD MODEL ONCE
# =============================================================================
print("Loading model... (this takes 10-20 seconds)")
start_time = time.time()

llm = Llama(
    model_path=MODEL_PATH,
    n_gpu_layers=N_GPU_LAYERS,  # GPU acceleration
    n_ctx=N_CTX,                 # Context window  
    n_batch=N_BATCH,             # Batch size
    verbose=False,               # Set to True if you want to see what's happening
)

print(f"✓ Model loaded in {time.time() - start_time:.1f} seconds")

# =============================================================================
# SIMPLE KPI EXTRACTION
# =============================================================================

def extract_kpis(text: str) -> List[Dict]:
    """
    Extract KPIs from a single sentence
    Simple, no async, no complexity
    """
    
    # Simple, direct prompt
    prompt = f"""Extract financial numbers from this sentence and output JSON array.

Sentence: {text}

Output format:
[{{"category": "revenue", "value": 92, "unit": "percent", "year": 2018}}]

Categories: revenue, earnings, debt, expenses, employees, assets, ratios
Units: percent, USD_millions, USD_billions, count, BOE, ratio

Output JSON array only:"""
    
    # Generate response
    try:
        response = llm(
            prompt,
            max_tokens=256,
            temperature=TEMPERATURE,
            stop=["```", "\n\n"],  # Stop at code blocks or double newline
            echo=False,  # Don't echo the prompt
        )
        
        # Extract text from response
        output = response['choices'][0]['text'].strip()
        
        # Clean up JSON
        # Try to find JSON array
        json_match = re.search(r'\[.*?\]', output, re.DOTALL)
        if json_match:
            json_str = json_match.group(0)
            # Basic cleanup
            json_str = json_str.replace("'", '"')
            json_str = re.sub(r',\s*}', '}', json_str)
            json_str = re.sub(r',\s*]', ']', json_str)
            
            try:
                data = json.loads(json_str)
                return data if isinstance(data, list) else [data]
            except json.JSONDecodeError:
                pass
    
    except Exception as e:
        print(f"  Error: {str(e)[:100]}")
    
    return []

# =============================================================================
# TEST ON YOUR SENTENCES
# =============================================================================

def main():
    """Test extraction on your 7 sentences"""
    
    test_sentences = [
        "For 2018, we estimate that mobile advertising revenue represented approximately 92% of total advertising revenue, as compared with approximately 88% in 2017.",
        "Realization of deferred tax assets associated with net operating loss and credit carryforwards is dependent upon generating sufficient taxable income prior to their expiration in the appropriate tax jurisdiction.",
        "A reduction of 85 million BOE was recorded in Canada, primarily from commodity price effects at Kaybob Duvernay.",
        "2018 compared to 2017 The Other segment reported net income of $14 million for the year ended December 31, 2018, compared to net loss of $71 million for the year ended December 31, 2017.",
        "Future policy benefits for individual life insurance and annuity policies consider crediting rates ranging from 2 1/2% to 6% for life insurance and 2% to 9 1/2% for annuities.",
        "The following table presents restructuring activity for the years ended June 30, 2019 and 2018: Separation Costs Employee separation charges for the years ended June 30, 2019 and 2018 relate to severance packages for approximately 1,810 and 2,720 employees, respectively.",
        "In May 2017, we issued $750.0 million of 2.35 percent fixed-rate notes due in May 2022, $750.0 million of 3.10 percent fixed-rate notes due in May 2027, and $750.0 million of 3.95 percent fixed-rate notes due in May 2047, with interest to be paid semi-annually."
    ]
    
    print("\n" + "="*70)
    print("Testing KPI Extraction - Simple & Direct")
    print("="*70)
    
    results = []
    total_time = 0
    
    for i, sentence in enumerate(test_sentences, 1):
        print(f"\n[{i}/7] Processing...")
        print(f"Text: {sentence[:80]}...")
        
        start = time.time()
        kpis = extract_kpis(sentence)
        elapsed = time.time() - start
        total_time += elapsed
        
        results.append(kpis)
        
        if kpis:
            print(f"✓ Found {len(kpis)} KPIs in {elapsed:.1f}s:")
            for kpi in kpis:
                cat = kpi.get('category', '?')
                val = kpi.get('value', '?')
                unit = kpi.get('unit', '?')
                year = kpi.get('year', '')
                year_str = f" ({year})" if year else ""
                print(f"  • {cat}: {val} {unit}{year_str}")
        else:
            print(f"✗ No KPIs found ({elapsed:.1f}s)")
    
    # Summary
    print("\n" + "="*70)
    print("SUMMARY")
    print("="*70)
    
    total_kpis = sum(len(r) for r in results)
    successful = sum(1 for r in results if r)
    
    print(f"Sentences processed: {len(test_sentences)}")
    print(f"Successful extractions: {successful}/{len(test_sentences)}")
    print(f"Total KPIs found: {total_kpis}")
    print(f"Average time per sentence: {total_time/len(test_sentences):.1f}s")
    print(f"Total processing time: {total_time:.1f}s")
    
    # Extrapolation
    print(f"\nExtrapolated for 1M sentences:")
    print(f"  Time: {(1_000_000 * total_time / len(test_sentences)) / 3600:.1f} hours")
    print(f"  With 10 parallel processes: {(1_000_000 * total_time / len(test_sentences)) / 3600 / 10:.1f} hours")

if __name__ == "__main__":
    main()

# =============================================================================
# BONUS: Batch Processing Function for Production
# =============================================================================

def process_batch(sentences: List[str], batch_size: int = 10) -> List[List[Dict]]:
    """
    Process multiple sentences in batches
    Simple sequential processing - no async needed!
    """
    results = []
    
    for i in range(0, len(sentences), batch_size):
        batch = sentences[i:i+batch_size]
        print(f"Processing batch {i//batch_size + 1}...")
        
        for sent in batch:
            kpis = extract_kpis(sent)
            results.append(kpis)
    
    return results

# =============================================================================
# ALTERNATIVE: Using Grammar for Guaranteed JSON
# =============================================================================

def extract_with_grammar(text: str) -> List[Dict]:
    """
    Use grammar to force valid JSON output
    This guarantees parseable JSON but may be more restrictive
    """
    
    # JSON grammar (simplified)
    json_grammar = r"""
    root   ::= array
    array  ::= "[" ws (object ("," ws object)*)? ws "]"
    object ::= "{" ws string ":" ws value ("," ws string ":" ws value)* ws "}"
    string ::= "\"" ([^"\\] | "\\" .)* "\""
    value  ::= string | number
    number ::= "-"? [0-9]+ ("." [0-9]+)?
    ws     ::= [ \t\n]*
    """
    
    prompt = f"Extract KPIs from: {text}\nOutput JSON array:"
    
    response = llm(
        prompt,
        max_tokens=256,
        temperature=0.1,
        grammar=json_grammar,  # Force JSON structure
    )
    
    try:
        return json.loads(response['choices'][0]['text'])
    except:
        return []

Loading model... (this takes 10-20 seconds)


llama_context: n_ctx_per_seq (4096) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


✓ Model loaded in 0.9 seconds

Testing KPI Extraction - Simple & Direct

[1/7] Processing...
Text: For 2018, we estimate that mobile advertising revenue represented approximately ...
✓ Found 1 KPIs in 11.9s:
  • revenue: 92 percent (2018)

[2/7] Processing...
Text: Realization of deferred tax assets associated with net operating loss and credit...
✗ No KPIs found (4.5s)

[3/7] Processing...
Text: A reduction of 85 million BOE was recorded in Canada, primarily from commodity p...
✗ No KPIs found (4.2s)

[4/7] Processing...
Text: 2018 compared to 2017 The Other segment reported net income of $14 million for t...
✓ Found 2 KPIs in 20.4s:
  • earnings: 14 USD_millions (2018)
  • earnings: -71 USD_millions (2017)

[5/7] Processing...
Text: Future policy benefits for individual life insurance and annuity policies consid...
✓ Found 4 KPIs in 25.3s:
  • revenue: 2.5 percent
  • revenue: 6 percent
  • revenue: 2 percent
  • revenue: 9.5 percent

[6/7] Processing...
Text: The following table pre

````

- 564,551 sentences at 5 seconds each = 783 hours (still too long!)
- 564,551 sentences at 3 seconds each = 470 hours (still impractical)

- The core problem: 564k sentences is still way too many for sequential LLM processing.
- ROBERTA will miss many non-standard KPIs (reserves, coupons, headcount, policy rates, BOE, regulatory capital).

In [ ]:
# """
# Financial KPI Extractor - 
# Fixes context size, asyncio issues, and server arguments for Windows llama.cpp
# """

# import os, time, json, subprocess, requests, re, asyncio, aiohttp
# from typing import List, Dict, Optional, Tuple
# from pathlib import Path
# import sys
# import nest_asyncio  # Add this to requirements

# # Fix for Jupyter/IPython environments
# try:
#     import nest_asyncio
#     nest_asyncio.apply()
# except ImportError:
#     print("Warning: nest_asyncio not found. Install with: pip install nest-asyncio")

# # =============================================================================
# # CONFIGURATION - Q5_K_M Model
# # =============================================================================
# LLAMA_DIR = r"C:\llama_server\llama_cpp_b6814"
# MODEL_PATH = r"C:\llama_server\models\qwen2p5_7b\Qwen2.5-7B-Instruct-Q5_K_M.gguf"

# HOST = "localhost"
# PORT = 8080
# BASE_URL = f"http://{HOST}:{PORT}"
# V1_URL = f"{BASE_URL}/v1"

# # Server parameters - Fixed for your build
# CTX_SIZE = 4096           
# N_BATCH = 512            
# PARALLEL = 8              # Reduced for stability
# GPU_LAYERS = 35           # Specific number instead of 999
# N_PREDICT = 256          
# TEMPERATURE = 0.1        

# # Async batching parameters
# MAX_CONCURRENT = 4        # Start conservative
# REQUEST_TIMEOUT = 30      

# # =============================================================================
# # SERVER MANAGEMENT - FIXED VERSION
# # =============================================================================

# def wait_for_server(base_url: str, timeout: int = 90) -> bool:
#     """Wait for server with proper checks"""
#     print(f"Waiting for server at {base_url}...")
#     start_time = time.time()
    
#     while time.time() - start_time < timeout:
#         try:
#             # Check health
#             response = requests.get(f"{base_url}/health", timeout=2)
#             if response.status_code == 200:
#                 print("✓ Server is healthy")
                
#                 # Try to get actual server info via completion endpoint
#                 try:
#                     # Send a test completion to verify context
#                     test_payload = {
#                         "prompt": "test",
#                         "max_tokens": 1,
#                         "temperature": 0
#                     }
#                     test_resp = requests.post(f"{base_url}/completion", json=test_payload, timeout=5)
#                     if test_resp.status_code == 200:
#                         print("✓ Completion endpoint working")
#                 except:
#                     pass
                
#                 # Check v1 endpoints
#                 try:
#                     models_resp = requests.get(f"{base_url}/v1/models", timeout=2)
#                     if models_resp.status_code == 200:
#                         print("✓ OpenAI v1 endpoints ready")
#                         return True
#                 except:
#                     pass
                    
#                 return True  # Health check passed, that's enough
#         except requests.exceptions.RequestException:
#             pass
#         time.sleep(1)
    
#     return False

# def start_llama_server(
#     model_path: str,
#     llama_dir: str = LLAMA_DIR,
#     port: int = PORT,
#     ctx_size: int = CTX_SIZE,
#     n_batch: int = N_BATCH,
#     parallel: int = PARALLEL,
#     gpu_layers: int = GPU_LAYERS
# ) -> subprocess.Popen:
#     """Start llama-server with CORRECT arguments for Windows build b6814"""
    
#     exe_path = Path(llama_dir) / "llama-server.exe"
#     if not exe_path.exists():
#         raise FileNotFoundError(f"llama-server.exe not found at {exe_path}")
    
#     model_path = Path(model_path)
#     if not model_path.exists():
#         raise FileNotFoundError(f"Model not found at {model_path}")
    
#     # FIXED: Use the CORRECT argument format for your build
#     # Your build uses different flags than documented
#     cmd = [
#         str(exe_path),
#         "-m", str(model_path),
#         "--host", "0.0.0.0",
#         "--port", str(port),
#         "--ctx-size", str(ctx_size),     # Try --ctx-size
#         "--batch-size", str(n_batch),    # Try --batch-size
#         "--n-parallel", str(parallel),   # Try --n-parallel
#         "--n-gpu-layers", str(gpu_layers),  # Full form
#         "--cont-batching",
#     ]
    
#     print("="*70)
#     print("Starting llama.cpp server with Qwen2.5-7B-Instruct Q5_K_M")
#     print("="*70)
#     print(f"\nModel: {model_path.name}")
#     print(f"Size: {model_path.stat().st_size/1e9:.2f} GB")
#     print(f"Target Context: {ctx_size} tokens")
#     print(f"Parallel slots: {parallel}")
#     print(f"\nCommand:")
#     print(" ".join(cmd))
#     print()
    
#     # Start process with proper error handling
#     try:
#         process = subprocess.Popen(
#             cmd,
#             stdout=subprocess.PIPE,
#             stderr=subprocess.PIPE,  # Capture stderr separately
#             text=True,
#             bufsize=1,
#             creationflags=subprocess.CREATE_NEW_PROCESS_GROUP if sys.platform == "win32" else 0
#         )
#     except Exception as e:
#         print(f"Failed to start server: {e}")
#         raise
    
#     # Monitor startup
#     print("Server startup output:")
#     for _ in range(30):
#         line = process.stdout.readline()
#         if line:
#             print(f"  {line.rstrip()}")
#             # Check for error indicators
#             if "error:" in line.lower() or "invalid" in line.lower():
#                 print(f"\n⚠ Detected error in output: {line}")
    
#     # Check if process is still running
#     if process.poll() is not None:
#         print("\n❌ Server exited during startup")
#         stderr = process.stderr.read()
#         if stderr:
#             print(f"Error output: {stderr}")
#         raise RuntimeError("Server failed to start")
    
#     # Wait for ready
#     if not wait_for_server(BASE_URL, timeout=90):
#         print("\nServer not responding properly")
#         process.terminate()
#         raise RuntimeError("Server failed to become ready")
    
#     print("\n✓ Server started successfully")
#     return process

# def stop_server(process: Optional[subprocess.Popen]):
#     """Gracefully stop the server"""
#     if process and process.poll() is None:
#         print("\nStopping server...")
#         if sys.platform == "win32":
#             process.terminate()
#         else:
#             process.terminate()
#         try:
#             process.wait(timeout=10)
#             print("✓ Server stopped")
#         except subprocess.TimeoutExpired:
#             print("⚠ Force killing server...")
#             process.kill()

# # =============================================================================
# # KPI EXTRACTION - Simplified and Robust
# # =============================================================================

# def clean_json_response(text: str) -> str:
#     """Extract and clean JSON from LLM response"""
#     # Remove markdown
#     text = re.sub(r'```(?:json)?\s*|\s*```', '', text)
    
#     # Find JSON array
#     array_match = re.search(r'\[.*?\]', text, re.DOTALL)
#     if array_match:
#         json_str = array_match.group(0)
#         # Fix common issues
#         json_str = json_str.replace("'", '"')  # Single to double quotes
#         json_str = re.sub(r',\s*]', ']', json_str)  # Trailing commas
#         json_str = re.sub(r',\s*}', '}', json_str)
#         return json_str
    
#     # Find JSON object
#     obj_match = re.search(r'\{.*?\}', text, re.DOTALL)
#     if obj_match:
#         return f"[{obj_match.group(0)}]"
    
#     return "[]"

# async def extract_kpis_async(
#     session: aiohttp.ClientSession,
#     text: str,
#     sentence_id: str,
#     semaphore: asyncio.Semaphore
# ) -> Tuple[str, List[Dict]]:
#     """Async KPI extraction with robust error handling"""
    
#     # Simplified prompt for better results
#     prompt = f"""Extract numbers from: "{text}"

# Output JSON array with: category, value, unit, year (if mentioned)
# Categories: revenue, earnings, debt, expenses, employees, ratios
# Units: percent, USD_millions, count, BOE

# Example: [{{"category":"revenue","value":92,"unit":"percent","year":2018}}]

# JSON only:"""
    
#     payload = {
#         "messages": [
#             {"role": "user", "content": prompt}
#         ],
#         "temperature": TEMPERATURE,
#         "max_tokens": N_PREDICT,
#         "stream": False,
#     }
    
#     async with semaphore:
#         try:
#             async with session.post(
#                 f"{V1_URL}/chat/completions",
#                 json=payload,
#                 timeout=aiohttp.ClientTimeout(total=REQUEST_TIMEOUT)
#             ) as response:
#                 if response.status == 200:
#                     result = await response.json()
#                     content = result.get("choices", [{}])[0].get("message", {}).get("content", "")
#                     cleaned = clean_json_response(content)
                    
#                     try:
#                         data = json.loads(cleaned) if cleaned else []
#                         if isinstance(data, dict):
#                             data = [data]
#                         return sentence_id, data if isinstance(data, list) else []
#                     except json.JSONDecodeError:
#                         return sentence_id, []
#                 else:
#                     error_text = await response.text()
#                     print(f"API error {response.status} for {sentence_id}: {error_text[:100]}")
#                     return sentence_id, []
                    
#         except asyncio.TimeoutError:
#             print(f"Timeout for {sentence_id}")
#             return sentence_id, []
#         except Exception as e:
#             print(f"Error for {sentence_id}: {str(e)[:100]}")
#             return sentence_id, []

# async def batch_extract_kpis(
#     sentences: List[Tuple[str, str]],
#     max_concurrent: int = MAX_CONCURRENT
# ) -> Dict[str, List[Dict]]:
#     """Process sentences in parallel with progress tracking"""
    
#     semaphore = asyncio.Semaphore(max_concurrent)
    
#     # Configure connection pool
#     connector = aiohttp.TCPConnector(limit=max_concurrent, force_close=True)
#     timeout = aiohttp.ClientTimeout(total=REQUEST_TIMEOUT)
    
#     async with aiohttp.ClientSession(connector=connector, timeout=timeout) as session:
#         tasks = [
#             extract_kpis_async(session, text, sid, semaphore)
#             for sid, text in sentences
#         ]
        
#         results = {}
#         completed = 0
#         total = len(tasks)
#         start_time = time.time()
        
#         print(f"\nProcessing {total} sentences with {max_concurrent} parallel workers...")
        
#         # Process with progress
#         for coro in asyncio.as_completed(tasks):
#             sid, kpis = await coro
#             results[sid] = kpis
#             completed += 1
            
#             # Progress bar
#             bar_length = 40
#             filled = int(bar_length * completed / total)
#             bar = "█" * filled + "░" * (bar_length - filled)
            
#             elapsed = time.time() - start_time
#             rate = completed / elapsed if elapsed > 0 else 0
            
#             print(f"\r[{bar}] {completed}/{total} ({completed/total*100:.0f}%) | "
#                   f"{rate:.1f} sent/s", end="", flush=True)
        
#         print()  # New line after progress bar
#         return results

# # =============================================================================
# # SYNCHRONOUS FALLBACK (for debugging)
# # =============================================================================

# def extract_kpis_sync(text: str, sentence_id: str) -> List[Dict]:
#     """Synchronous extraction for testing"""
#     prompt = f"""Extract financial numbers from: "{text}"
# Output JSON: [{{"category":"type","value":number,"unit":"unit"}}]"""
    
#     payload = {
#         "messages": [{"role": "user", "content": prompt}],
#         "temperature": 0.1,
#         "max_tokens": 150,
#     }
    
#     try:
#         response = requests.post(f"{V1_URL}/chat/completions", json=payload, timeout=30)
#         if response.status_code == 200:
#             content = response.json()["choices"][0]["message"]["content"]
#             cleaned = clean_json_response(content)
#             data = json.loads(cleaned) if cleaned and cleaned != "[]" else []
#             return data if isinstance(data, list) else []
#     except Exception as e:
#         print(f"Sync extraction error: {e}")
#     return []

# # =============================================================================
# # MAIN EXECUTION
# # =============================================================================

# def run_async_in_jupyter(coro):
#     """Helper to run async code in Jupyter/IPython"""
#     try:
#         # Try to get existing loop
#         loop = asyncio.get_running_loop()
#         # Create task and run
#         import nest_asyncio
#         nest_asyncio.apply()
#         return asyncio.run(coro)
#     except RuntimeError:
#         # No loop running, use asyncio.run
#         return asyncio.run(coro)

# def main():
#     """Main execution function"""
    
#     # Test sentences
#     test_sentences = [
#         "For 2018, we estimate that mobile advertising revenue represented approximately 92% of total advertising revenue, as compared with approximately 88% in 2017.",
#         "Realization of deferred tax assets associated with net operating loss and credit carryforwards is dependent upon generating sufficient taxable income prior to their expiration in the appropriate tax jurisdiction.",
#         "A reduction of 85 million BOE was recorded in Canada, primarily from commodity price effects at Kaybob Duvernay.",
#         "2018 compared to 2017 The Other segment reported net income of $14 million for the year ended December 31, 2018, compared to net loss of $71 million for the year ended December 31, 2017.",
#         "Future policy benefits for individual life insurance and annuity policies consider crediting rates ranging from 2 1/2% to 6% for life insurance and 2% to 9 1/2% for annuities.",
#         "The following table presents restructuring activity for the years ended June 30, 2019 and 2018: Separation Costs Employee separation charges for the years ended June 30, 2019 and 2018 relate to severance packages for approximately 1,810 and 2,720 employees, respectively.",
#         "In May 2017, we issued $750.0 million of 2.35 percent fixed-rate notes due in May 2022, $750.0 million of 3.10 percent fixed-rate notes due in May 2027, and $750.0 million of 3.95 percent fixed-rate notes due in May 2047, with interest to be paid semi-annually."
#     ]
    
#     print("="*70)
#     print("Financial KPI Extractor - FIXED VERSION")
#     print("Model: Qwen2.5-7B-Instruct Q5_K_M")
#     print("="*70)
    
#     server_process = None
    
#     try:
#         # Start server
#         server_process = start_llama_server(MODEL_PATH)
        
#         # Give it a moment to stabilize
#         time.sleep(2)
        
#         # Test with synchronous first
#         print("\n" + "="*70)
#         print("Testing with synchronous extraction first...")
#         print("="*70)
        
#         test_sid = "S001"
#         test_text = test_sentences[0]
#         print(f"\nTest sentence: {test_text[:80]}...")
        
#         test_result = extract_kpis_sync(test_text, test_sid)
#         if test_result:
#             print(f"✓ Sync test successful: {test_result}")
#         else:
#             print("⚠ Sync test returned no results")
        
#         # Prepare data for async
#         test_data = [(f"S{i:03d}", sent) for i, sent in enumerate(test_sentences, 1)]
        
#         print("\n" + "="*70)
#         print("Running async batch extraction...")
#         print("="*70)
        
#         # Run async with proper handling for Jupyter
#         try:
#             # Check if we're in Jupyter/IPython
#             get_ipython()  # This will raise NameError if not in IPython
#             print("Detected Jupyter/IPython environment")
            
#             # Use nest_asyncio approach
#             import nest_asyncio
#             nest_asyncio.apply()
#             results = asyncio.run(batch_extract_kpis(test_data, max_concurrent=MAX_CONCURRENT))
            
#         except NameError:
#             # Not in Jupyter, use normal asyncio
#             print("Running in standard Python environment")
#             results = asyncio.run(batch_extract_kpis(test_data, max_concurrent=MAX_CONCURRENT))
        
#         # Display results
#         print("\n" + "="*70)
#         print("EXTRACTION RESULTS")
#         print("="*70)
        
#         total_kpis = 0
#         sentences_with_kpis = 0
        
#         for i, (sid, sent) in enumerate(test_data, 1):
#             kpis = results.get(sid, [])
            
#             print(f"\n[{sid}] Sentence {i}:")
#             print(f"Text: {sent[:100]}...")
            
#             if kpis:
#                 sentences_with_kpis += 1
#                 total_kpis += len(kpis)
#                 for kpi in kpis:
#                     # Format KPI nicely
#                     cat = kpi.get('category', 'unknown')
#                     val = kpi.get('value', 'N/A')
#                     unit = kpi.get('unit', '')
#                     year = kpi.get('year', '')
#                     year_str = f" ({year})" if year else ""
#                     print(f"  ✓ {cat}: {val} {unit}{year_str}")
#             else:
#                 print("  ✗ No KPIs extracted")
        
#         # Summary
#         print("\n" + "="*70)
#         print("SUMMARY")
#         print("="*70)
#         print(f"Sentences processed: {len(test_data)}")
#         print(f"Sentences with KPIs: {sentences_with_kpis}/{len(test_data)} ({sentences_with_kpis/len(test_data)*100:.0f}%)")
#         print(f"Total KPIs extracted: {total_kpis}")
#         if len(test_data) > 0:
#             print(f"Average KPIs/sentence: {total_kpis/len(test_data):.1f}")
        
#     except KeyboardInterrupt:
#         print("\n\n⚠ Interrupted by user")
        
#     except Exception as e:
#         print(f"\n❌ Error: {e}")
#         import traceback
#         traceback.print_exc()
        
#     finally:
#         if server_process:
#             stop_server(server_process)
    
#     print("\n" + "="*70)
#     print("Session complete")
#     print("="*70)

# if __name__ == "__main__":
#     main()